<a href="https://colab.research.google.com/github/Dhruvilpatel04/Spotify_song_recomendation/blob/main/Spotify_songs_Reccomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/BDA_dataset/SpotifyFeatures.csv")
# data = data.sample(n=5000, random_state=3)
data = data.head(10000)

In [ ]:
data

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,1GM1o7Wgh3UhqgMf5Yqou6,Armandinho,Acústico (Ao Vivo),O Que Meu Pai Falou Pra Mim - Ao Vivo,42,252390,False,0.629,0.5810,...,-9.571,1,0.0420,0.5470,0.000018,0.6810,0.299,86.005,4,brazil
9996,9996,3UFZkGoRoqZD2gp0ayZlM7,Canção & Louvor,Profetizando Vida,Dependente,43,270791,False,0.647,0.4590,...,-6.737,0,0.0331,0.5860,0.000000,0.1900,0.201,135.824,4,brazil
9997,9997,0VYLCqCYrLo2aZ1mcdVDjH,Planet Hemp,Usuário,"Não Compre, Plante!",42,251493,False,0.652,0.7800,...,-8.433,1,0.0507,0.0195,0.033400,0.1860,0.656,112.680,4,brazil
9998,9998,5BtFkPust4wifo9KOThNGI,Cidade Negra;Lulu Santos,Quanto Mais Curtido Melhor,Sábado a Noite (feat. Lulu Santos),42,267306,False,0.773,0.9400,...,-5.303,0,0.0792,0.1930,0.000000,0.1900,0.887,97.135,4,brazil


,artists,track_name,album_name,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,time_signature,valence


In [ ]:
# Select the relevant features for recommendation
features = ['artists', 'track_name','album_name', 'duration_ms','popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
data = data[features]


In [ ]:
# Preprocess the data
data = data.dropna()
data = data.drop_duplicates(subset='track_name')


In [ ]:
# Convert the track_name column to lowercase
data['track_name'] = data['track_name'].str.lower()

In [ ]:
# Create a CountVectorizer object and fit it to the track_name column
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(data['track_name'])


In [ ]:
# Compute the cosine similarity matrix
cosine_sim = cosine_similarity( count_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
def get_recommendations(track_name, data, cosine_sim):
    # Make sure the track name is in the DataFrame
    if track_name not in data['track_name'].values:
        print(f'Track "{track_name}" not found in the dataset.')
        return pd.DataFrame()

    # Get the index of the track
    idx = data[data['track_name'] == track_name].index[0]

    # Get similarity scores for all tracks
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort tracks by similarity score in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top 10 similar tracks (excluding itself)
    track_indices = [i[0] for i in sim_scores if i[0] != idx][:10]

    # Get the data of the recommended tracks
    recommended_tracks = data.iloc[track_indices][['track_name', 'artists', 'album_name']]

    return recommended_tracks


In [ ]:
# random_track_name = data['track_name'].sample().iloc[0]
random_track_name = data.iloc[7][1]
# get recommendations for the random track
recommendations = get_recommendations(random_track_name, data, cosine_sim)

recommendations = pd.DataFrame(recommendations)
# print the recommendations
print(f'Recommended tracks for "{random_track_name}":\n')
recommendations


Recommended tracks for "i'm yours":



,track_name,artists,album_name
0,comedy,Gen Hoshino,Comedy
1,ghost - acoustic,Ben Woodward,Ghost (Acoustic)
2,to begin again,Ingrid Michaelson;ZAYN,To Begin Again
3,can't help falling in love,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...
4,hold on,Chord Overstreet,Hold On
5,days i will remember,Tyrone Wells,Days I Will Remember
6,say something,A Great Big World;Christina Aguilera,Is There Anybody Out There?
8,lucky,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.
9,hunger,Ross Copperman,Hunger
10,give me your forever,Zack Tabudlo,Episode


In [ ]:
data1 = pd.read_csv("/content/drive/MyDrive/BDA_dataset/SpotifyFeatures.csv")

sorted_songs = data1.sort_values(by='popularity', ascending=False)
def recommend_popular_songs(n=10):
    popular_songs = sorted_songs.head(n)
    recommended_songs = data1[['track_name','artists','album_name', 'popularity']]
    return recommended_songs

# Example usage: Recommend top 10 popular songs
recommended_songs = recommend_popular_songs(n=10)
recommended_songs = pd.DataFrame(recommended_songs )
recommended_songs

,track_name,artists,album_name,popularity
0,Comedy,Gen Hoshino,Comedy,73
1,Ghost - Acoustic,Ben Woodward,Ghost (Acoustic),55
2,To Begin Again,Ingrid Michaelson;ZAYN,To Begin Again,57
3,Can't Help Falling In Love,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,71
4,Hold On,Chord Overstreet,Hold On,82
...,...,...,...,...
113995,Sleep My Little Boy,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,21
113996,Water Into Light,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,22
113997,Miss Perfumado,Cesária Evora,Best Of,22
113998,Friends,Michael W. Smith,Change Your World,41


In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195816 sha256=97b0e498d01be7a373cc7da907a6df6df6e2c58fe951d6069e920d7f908f909f
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [ ]:
x = input("Enter Genre")
sorted_songs_jonours = data1[data1['track_genre']== x]
sorted_songs_jonours
sorted_songs = sorted_songs_jonours.sort_values(by='popularity', ascending=False)
sorted_songs
def recommend_popular_songs(n=10):
    popular_songs = sorted_songs.head(n)
    recommended_songs = data1[['track_name','artists','album_name', 'popularity']]
    return recommended_songs

# Example usage: Recommend top 10 popular songs
recommended_songs = recommend_popular_songs(n=10)
print(recommended_songs)

Enter Genrebrazil
                        track_name                 artists  \
0                           Comedy             Gen Hoshino   
1                 Ghost - Acoustic            Ben Woodward   
2                   To Begin Again  Ingrid Michaelson;ZAYN   
3       Can't Help Falling In Love            Kina Grannis   
4                          Hold On        Chord Overstreet   
...                            ...                     ...   
113995         Sleep My Little Boy           Rainy Lullaby   
113996            Water Into Light           Rainy Lullaby   
113997              Miss Perfumado           Cesária Evora   
113998                     Friends        Michael W. Smith   
113999                   Barbincor           Cesária Evora   

                                               album_name  popularity  
0                                                  Comedy          73  
1                                        Ghost (Acoustic)          55  
2                    